In [46]:
from zarrnii import ZarrNii, ZarrNiiAtlas

img = ZarrNii.from_ome_zarr('/nfs/trident3/lightsheet/prado/mouse_app_lecanemab_batch3/bids/sub-A164F5/micr/sub-A164F5_sample-brain_acq-imaris4x_SPIM.ome.zarr', channel_labels=['CD31'],level=2,downsample_near_isotropic=True, chunks=(1,500,500,500), rechunk=True)
img

ZarrNii(
  name='image', shape=(1, 1200, 3136, 1824), dims=['c', 'z', 'y', 'x'],
  axes_order='ZYX', xyz_orientation='RPI',
  scale={'c': 1, 'z': 0.0055, 'y': 0.0065, 'x': 0.0065},
  dtype=uint16, chunksize=(1, 500, 500, 500)
)

In [12]:
img.to_nifti('CD31.nii')


'CD31.nii'

In [13]:

atlas = ZarrNiiAtlas.from_files('/nfs/trident3/lightsheet/prado/mouse_app_lecanemab_ki3/derivatives/spimquant_aae813e/sub-AS134F3/micr/sub-AS134F3_sample-brain_acq-imaris4x_seg-all_from-ABAv3_level-5_desc-deform_dseg.nii.gz',
                        '/nfs/trident3/lightsheet/prado/mouse_app_lecanemab_ki3/derivatives/spimquant_aae813e/tpl-ABAv3/seg-all_tpl-ABAv3_dseg.tsv')

In [14]:
atlas


ZarrNiiAtlas(ngff_image=NgffImage(data=dask.array<getitem, shape=(1, 212, 312, 192), dtype=float64, chunksize=(1, 212, 312, 192), chunktype=numpy.ndarray>, dims=['c', 'x', 'y', 'z'], scale={'x': 0.052000001072883606, 'y': 0.052000001072883606, 'z': 0.03521599993109703}, translation={'x': np.float64(-0.0), 'y': np.float64(-0.0), 'z': np.float64(0.0)}, name='nifti_image_/nfs/trident3/lightsheet/prado/mouse_app_lecanemab_ki3/derivatives/spimquant_aae813e/sub-AS134F3/micr/sub-AS134F3_sample-brain_acq-imaris4x_seg-all_from-ABAv3_level-5_desc-deform_dseg.nii.gz', axes_units=None, axes_orientations=None, computed_callbacks=[]), axes_order='XYZ', xyz_orientation='RPI', _omero=None, labels_df=      index                                name abbreviation   color
0         0                           left root       root-L  000000
1         1  left Basic cell groups and regions       grey-L  BFDAE3
2         2                       left Cerebrum         CH-L  B0F0FF
3         3                left Cerebral cortex        CTX-L  B0FFB8
4         4                 left Cortical plate      CTXpl-L  70FF70
...     ...                                 ...          ...     ...
2649  11322    right intraparafloccular fissure        ipf-R  A9A9AA
2650  11323             right paramedian sulcus        pms-R  A9A9A9
2651  11324          right parafloccular sulcus        pfs-R  AAAAA8
2652  11325         right Interpeduncular fossa        IPF-R  A9A8A9
2653  11326                        right retina     retina-R  7F2E7E

[2654 rows x 4 columns], label_column='index', name_column='name', abbrev_column='abbreviation')

In [15]:
cropped = img.crop_with_bounding_box(*atlas.get_region_bounding_box(regex='Hipp'),ras_coords=True)


In [16]:
cropped

ZarrNii(
  name='image', shape=(1, 231, 232, 316), dims=['c', 'z', 'y', 'x'],
  axes_order='ZYX', xyz_orientation='RPI',
  scale={'c': 1, 'z': 0.022, 'y': 0.026, 'x': 0.026},
  dtype=uint16, chunksize=(1, 1, 232, 316)
)

In [56]:
fullresimg = ZarrNii.from_ome_zarr('/nfs/trident3/lightsheet/prado/mouse_app_lecanemab_batch3/bids/sub-A164F5/micr/sub-A164F5_sample-brain_acq-imaris4x_SPIM.ome.zarr', channel_labels=['CD31'],level=0, downsample_near_isotropic=True,chunks=(1,300,300,300), rechunk=True)


In [63]:
croppedfullres = fullresimg.crop_with_bounding_box(*atlas.get_region_bounding_box(regex='right Field CA1'),ras_coords=True)
croppedfullres


ZarrNii(
  name='image', shape=(1, 1665, 896, 1360), dims=['c', 'z', 'y', 'x'],
  axes_order='ZYX', xyz_orientation='RPI',
  scale={'c': 1, 'z': 0.00275, 'y': 0.00325, 'x': 0.00325},
  dtype=uint16, chunksize=(1, 300, 300, 300)
)

In [78]:
_,Z,Y,X = croppedfullres.shape
step = 600
for z_ in range(Z//step+1):
    for y_ in range(Y//step+1):
        for x_ in range(X//step+1):
            dg_slice = (croppedfullres
                        .crop_with_bounding_box((z_*step, y_*step, x_*step),(min((1+z_)*step,Z), min((y_+1)*step,Y), min((x_+1)*step,X))))
            dg_slice.to_nifti(f'CD31_rCA1{(z_*step, y_*step, x_*step),(min((1+z_)*step,Z), min((y_+1)*step,Y), min((x_+1)*step,X))}.nii.gz')
            print((z_*step, y_*step, x_*step),(min((1+z_)*step,Z), min((y_+1)*step,Y), min((x_+1)*step,X),dg_slice.shape))

(0, 0, 0) (600, 600, 600, (1, 600, 600, 600))
(0, 0, 600) (600, 600, 1200, (1, 600, 600, 600))
(0, 0, 1200) (600, 600, 1360, (1, 160, 600, 600))
(0, 600, 0) (600, 896, 600, (1, 600, 296, 600))
(0, 600, 600) (600, 896, 1200, (1, 600, 296, 600))
(0, 600, 1200) (600, 896, 1360, (1, 160, 296, 600))
(600, 0, 0) (1200, 600, 600, (1, 600, 600, 600))
(600, 0, 600) (1200, 600, 1200, (1, 600, 600, 600))
(600, 0, 1200) (1200, 600, 1360, (1, 160, 600, 600))
(600, 600, 0) (1200, 896, 600, (1, 600, 296, 600))
(600, 600, 600) (1200, 896, 1200, (1, 600, 296, 600))
(600, 600, 1200) (1200, 896, 1360, (1, 160, 296, 600))
(1200, 0, 0) (1665, 600, 600, (1, 600, 600, 160))
(1200, 0, 600) (1665, 600, 1200, (1, 600, 600, 160))
(1200, 0, 1200) (1665, 600, 1360, (1, 160, 600, 160))
(1200, 600, 0) (1665, 896, 600, (1, 600, 296, 160))
(1200, 600, 600) (1665, 896, 1200, (1, 600, 296, 160))
(1200, 600, 1200) (1665, 896, 1360, (1, 160, 296, 160))


In [41]:
from dask.diagnostics import ProgressBar

with ProgressBar():
    ZarrNii.from_ome_zarr('/nfs/trident3/lightsheet/prado/mouse_app_lecanemab_batch3/bids/sub-A164F5/micr/sub-A164F5_sample-brain_acq-imaris4x_SPIM.ome.zarr', channel_labels=['CD31'],level=1, downsample_near_isotropic=True).crop_with_bounding_box(*atlas.get_region_bounding_box(regex='right Agranular insular area, posterior part'),ras_coords=True).to_nifti('CD31_lhipp.nii.gz')

[########################################] | 100% Completed | 202.77 ms
[########################################] | 100% Completed | 326.29 ms
[########################################] | 100% Completed | 202.80 ms
[########################################] | 100% Completed | 101.43 ms
[########################################] | 100% Completed | 20.49 s


In [49]:
segmented = croppedfullres.segment_otsu(nbins=256)
segmented.to_nifti('seg.nii.gz')

'seg.nii.gz'